# Cryptohack 
## [ELLIPTIC CURVES](https://cryptohack.org/courses/elliptic/course_details/)

Writeup by Jupyter notebook

<img src="img/スクリーンショット 2023-05-04 14.13.32.png" alt="official">
crypto{Abelian}

任意の点 P に関して、 P + O = O + P = P

O 以外の2点 P(x_p, y_p), Q(x_q, y_q) に関して、 x_p = x_q かつ y_p = -y_q のとき、 P + Q = O

```
print(-6936%9739)
# 2803
```
crypto{8045,2803}

Algorithm for the addition of two points: P + Q

(a) If P = O, then P + Q = Q.<br>
(b) Otherwise, if Q = O, then P + Q = P.<br>
(c) Otherwise, write P = (x1, y1) and Q = (x2, y2).<br>
(d) If x1 = x2 and y1 = −y2, then P + Q = O.<br>
(e) Otherwise:<br>
&emsp;(e1) if P ≠ Q: λ = (y2 - y1) / (x2 - x1)<br>
&emsp;(e2) if P = Q: λ = (3x12 + a) / 2y1<br>
(f) x3 = λ2 − x1 − x2,     y3 = λ(x1 −x3) − y1<br>
(g) P + Q = (x3, y3)<br>


In [78]:
def ec_add(p, a, P, Q):
    zero = [0, 0]

    if P[0] == Q[0] and P[1] == -Q[1] % p:
        return zero
    elif P == zero:
        return Q
    elif Q == zero:
        return P
    elif Q == P:
        lamb = (pow((2*P[1]), -1, p) * (3*(P[0]**2) + a)) % p
    else:
        lamb = (pow(Q[0]-P[0], -1, p)*(Q[1]-P[1])) % p
    x = (lamb**2 - P[0] - Q[0]) % p
    # Q = (lamb**2 - Q1 - Q2) % p
    return [x, (lamb*(P[0]-x) - P[1]) % p]


p = 9739
a = 497
b = 1768
P = [493, 5564]
Q = [1539, 4742]
R = [4403, 5202]
print(ec_add(p, a, ec_add(p, a, P, P), ec_add(p, a, Q, R)))


[4215, 2162]


Double and Add algorithm for the scalar multiplication of point P by n

Input: P in E(Fp) and an integer n > 0<br>
1. Set Q = P and R = O.<br>
2. Loop while n > 0.<br>
&emsp;3. If n ≡ 1 mod 2, set R = R + Q.<br>
&emsp;4. Set Q = 2 Q and n = ⌊n/2⌋.<br>
&emsp;5. If n > 0, continue with loop at Step 2.<br>
6. Return the point R, which equals nP.<br>

In [58]:
import copy


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = [0, 0]
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 7863
P = [2339, 2213]
print(ec_scalar(p, a, P, n))


[9467, 2742]


In [107]:
from math import gcd
import hashlib


def legendre_symbol(a, p):
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y


p = 9739
a = 497
b = 1768
G = [1804, 5368]
QA = [815, 3190]
nB = 1829

sha1 = hashlib.sha1()
data = str(ec_scalar(p, a, QA, nB)[0])
sha1.update(data.encode("utf-8"))
print(sha1.hexdigest())


80e5212754a824d3a4aed185ace4f9cac0f908bf


In [106]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii', 'ignore'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii', 'ignore')
    else:
        return plaintext.decode('ascii', 'ignore')


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y

p = 9739
a = 497
b = 1768
G = [1804, 5368]
q_x = 4726
nB = 6534

data = str(ec_scalar(p, a, [q_x, ec(p, a, b, q_x)], nB)[0])
iv = 'cd9da9f1c60925922377ea952afc212c'
ciphertext = 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'

print(decrypt_flag(data, iv, ciphertext))


crypto{3ff1c1ent_k3y_3xch4ng3}


In [109]:
# Montgomery curve
def MontgomeryCurve(p, a, k):
    k = bin(k)[:2]
    print(k)
    R[0], R[1] = P, P[2]
    for i 
a = 486662
p = 2**255 - 19
Gx = 9
print(MontgomeryCurve(p, a, Gx))


KeyboardInterrupt: 

In [50]:
import copy
from math import gcd

def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec_get_0(p, a, P):
    return ec_add(p, a, P, [P[0], -P[1] % p])


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = ec_get_0(p, a, P)
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 1337
X = [5323, 5438]
P = [2339, 2213]
# print(ec_get_o(p,a,b,P))
print(X)
print(ec_add(p, a, X, ec_get_0(p, a, X)))
print(ec_scalar(p, a, X, n))


[5323, 5438]


ValueError: base is not invertible for the given modulus

In [101]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, squares, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9)/ (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 5
y1 = 7
a = 9
b = 1
x2=0
squares = SquareRoots(p)
print(ec_add(p, squares ,x1,y1,x1,y1))
print(ec_add(p, squares ,x1,y1,26,50))

while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


(26.0, 50.0)
(34.192743764172334, 55.22438181621857)


In [104]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False

# 楕円曲線の足し算


def ec_add(p, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9) / (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 9739
x1 = 8045
y1 = 6936
a = 497
b = 1768
x2 = 0
squares = SquareRoots(p)
x0 = 0
y0 = checkRoot(squares, (x0**3+a*x0+b) % p)
print(x0,y0)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b) % p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            if x3 == x0 and y3 == y0:
                print(x2, y2)
                break
    x2 += 1


0 4418


In [88]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, x1, y1, x2, y2):
    lamb = (y2-y1)/(x2-x1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 8
y1 = 6936
a = 497
b = 1768
a = b = x2 = 0
x2=2
squares = SquareRoots(p)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


10.760000000242144 45.57599966525959 3 24
15.0 33.0 4 8
21.444444444961846 19.2962951014415 5 8
56.0 54.0 9 27
56.0625 46.234375 12 9
52.0 46.0 13 1
21.694444444729015 24.671296624739625 14 11
56.61224489787128 55.28862965061853 15 9
43.890625 11.712890625 16 3
21.859504132240545 3.8084147315130394 19 24
36.0625 5.109375 20 3
6.0 6.0 22 20
1.031141868501436 42.93568084237995 25 3
37.07756232688553 44.64149293471928 27 23
57.25591715975315 25.913006369617506 34 9
6.938775510207051 0.18075801822578796 36 28
13.613943808530166 49.7283743406615 39 24
34.44444444444525 2.7037037038735434 41 28
29.34602076124429 29.03175249310334 42 20
34.8991964937959 38.15268592275879 45 28
40.26939058172138 32.481502406313666 46 23
28.134779750162124 60.376860701906025 47 1
26.265625 8.236328125 48 11
7.2498512790043605 13.517607116027648 49 23
26.933884297523036 10.226145755445486 52 8
36.00390625 22.812255859375 56 27
52.0 0.0 57 27
34.42239999999947 12.746367999926406 58 20
38.068417159767705 57.303631

In [10]:
# Y 2 = X 3 + 497 X + 1768、p: 9739
# P(8045,6936)
a = b = x = y = 0
while True:
    x += 1
    if x ** 3 + 495 * x + 1768 % 9739 == 8045:
        y = (x ** 3 + 495 * x + 1768 % 9739) ** 0.5 
        if y ** 0.5 % 9739 == 6936:
            print(x, y)


KeyboardInterrupt: 